<a href="https://colab.research.google.com/github/kjg331/kjg331-chat_with_your_data_Llama2-Langchain/blob/main/Chat_with_newsweb_Mistral_streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Dependencies

I used Mistral chat model and embeddings and a Chroma vector store in this walkthrough, but everything shown here works with any [ChatModel](/docs/modules/model_io/chat/) or [LLM](/docs/modules/model_io/llms/), [Embeddings](/docs/modules/data_connection/text_embedding/), and [VectorStore](/docs/modules/data_connection/vectorstores/) or [Retriever](/docs/modules/data_connection/retrievers/).

Specifically this code can be used in Colab, however need to use CPU to drive the llm. If you have enough GPU or Colab+ higly recommended to use the GPU version

I'll use the following packages:

In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai chromadb bs4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.6/803.6 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 73.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 43.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.38-cp310-cp310-manylinux_2_35_x86_64.whl size=9660682 sha256=e77fa8268b2c6291d908cd02c7f21e4b4f204888de30c718caed9fed5c4614e8
  Stored in directory: /root/.cache/pip/wheels/eb/58/77/20d3d9a235b4930050fbcde1ad4f0a4d054644269e801b08aa
Successfully built llama-cpp-python


In [3]:
# For download the models
!pip install huggingface_hub

In [4]:
%pip install --upgrade --quiet  langchain sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.1 MB/s eta 0:00:00


In [5]:
!pip install -U docarray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.2/270.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.0 MB/s eta 0:00:00


In [6]:
!pip install googletrans==4.0.0rc1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.5 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17396 sha256=518eccc43e25474ed62af73ff598c4a2c5033d92c40ec06e4fb8aca85fc6649b
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.

In [7]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q5_K_M.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q5_K_M.gguf:   0%|          | 0.00/5.13G [00:00<?, ?B/s]

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
from langchain import hub
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import WebBaseLoader
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain import LlamaCpp

import bs4

In [19]:
from googletrans import Translator
translator = Translator()

def ko_trans(sentence):
  if translator.detect(sentence).lang != "ko":
    return  translator.translate(sentence, dest='ko').text
  else:
    return sentence

In [10]:
# Load the LLM Model

llm = None
llm = LlamaCpp(model_path= model_path,
               max_tokens=4096,
               n_threads=2,
               n_gpu_layers=40,
               n_batch=16,
               callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
               n_ctx=4096, # Context window
               verbose = True,
               temperature = 0.1,
               repeat_penalty = 1.2
               )

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '1000000.000000', 'llama.context_length': '32768', 'general.na

In [11]:
def load_db(file, chain_type, k):
    # load documents
    loader = WebBaseLoader(
    file,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("media_end_head_title", "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME", "newsct_article _article_body")
        )
    ),
)
    documents = loader.load()

    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)

    # define embedding
    embeddings =  HuggingFaceEmbeddings(
                                        model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
                                        model_kwargs={'device': 'cpu'},
                                        encode_kwargs={'normalize_embeddings': True}
    )

    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)

    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})

    # create a chatbot chain. Memory is managed externally.
    qa = None
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa

In [ ]:
def get_answer(question, chat_history):

In [12]:
file_path = "https://n.news.naver.com/mnews/article/003/0012345220?sid=101"

qa_bot = load_db(file_path)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/336k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/967k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
chat_history = []

question_1 = "무슨 내용에 관한 뉴스입니까?"
reuslt_1 = qa_bot({"question": question_1, "chat_history":chat_history})
chat_history.extend([(question_1, reuslt_1["answer"])])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


 Yes, it appears to be a news article based on its format and content.

In [14]:
question_2 = "Summarize the content of the news"
reuslt_2 = qa_bot({"question": question_2, "chat_history":chat_history})
chat_history.extend([(question_2, reuslt_2["answer"])])

Llama.generate: prefix-match hit


 Could you please summarize the content of this news article for me?

Llama.generate: prefix-match hit


 This news article discusses a company called "Samsung Electronics Co., Ltd." and its plans for a new type of memory called "High Bandwidth Memory (HBM)" for use in graphics processing units (GPUs) and other high-performance computing applications. The article also mentions other types of memory such as "DRAM (Dynamic Random Access Memory)" and "HBM2 (High Bandwidth Memory version 2)" and their respective uses and advantages in various computing applications. The article also provides some background information on Samsung Electronics and its role in the semiconductor industry. Overall, the article provides valuable information on Samsung Electronics and its plans for High Bandwidth Memory (HBM) for use in high-performance computing applications such as graphics processing units (GPUs). The article also provides some background information on Samsung Electronics and its role in the semiconductor industry.

In [16]:
reuslt_2['answer']

' This news article discusses a company called "Samsung Electronics Co., Ltd." and its plans for a new type of memory called "High Bandwidth Memory (HBM)" for use in graphics processing units (GPUs) and other high-performance computing applications. The article also mentions other types of memory such as "DRAM (Dynamic Random Access Memory)" and "HBM2 (High Bandwidth Memory version 2)" and their respective uses and advantages in various computing applications. The article also provides some background information on Samsung Electronics and its role in the semiconductor industry. Overall, the article provides valuable information on Samsung Electronics and its plans for High Bandwidth Memory (HBM) for use in high-performance computing applications such as graphics processing units (GPUs). The article also provides some background information on Samsung Electronics and its role in the semiconductor industry.'

In [22]:
answer_2_ko = ko_trans(reuslt_2['answer'])
print(answer_2_ko)

이 뉴스 기사는 "Samsung Electronics Co., Ltd."라는 회사에 대해 설명합니다.그래픽 처리 장치 (GPU) 및 기타 고성능 컴퓨팅 애플리케이션에 사용하기위한 "High Bandwidth Memory (HBM)"라는 새로운 유형의 메모리에 대한 계획.이 기사는 또한 "DRAM (Dynamic Random Access Memory)"및 "HBM2 (높은 대역폭 메모리 버전 2)"와 같은 다른 유형의 메모리와 다양한 컴퓨팅 애플리케이션에서 각각의 용도 및 장점을 언급합니다.이 기사는 또한 삼성 전자 장치와 반도체 산업에서의 역할에 대한 배경 정보를 제공합니다.전반적 으로이 기사는 Samsung Electronics에 대한 귀중한 정보와 GPU (Graphics Processing Units)와 같은 고성능 컴퓨팅 애플리케이션에 사용하기위한 고 대역폭 메모리 (HBM) 계획에 대한 정보를 제공합니다.이 기사는 또한 삼성 전자 장치와 반도체 산업에서의 역할에 대한 배경 정보를 제공합니다.


In [17]:
question_3 = "Is the news is only about the Samsung Electronics? if other please also explian me about that"
reuslt_3 = qa_bot({"question": question_3, "chat_history":chat_history})
chat_history.extend([(question_3, reuslt_3["answer"])])

Llama.generate: prefix-match hit


 What other companies are mentioned in the news article besides Samsung Electronics?

Llama.generate: prefix-match hit


 The news article mentions several other companies besides Samsung Electronics. Some of these companies include:
- Intel (IT)
- Micron Technology (DRAM)
- Hynix Semiconductor (HBM)
These companies are mentioned in relation to their roles in the semiconductor industry and their partnerships with Samsung Electronics and other companies mentioned in the article.

In [21]:
answer_3_ko = ko_trans(reuslt_3['answer'])
print(answer_3_ko)

뉴스 기사는 삼성 전자 제품 외에 다른 여러 회사에 언급되어 있습니다.이들 회사 중 일부는 다음과 같습니다.
- 인텔 (IT)
- 미크론 기술 (DRAM)
-Hynix 반도체 (HBM)
이 회사들은 반도체 산업에서의 역할과 관련하여 언급되어 있으며,이 기사에 언급 된 삼성 전자 및 기타 회사와의 파트너십.


In [24]:
question_4 = "삼성과의 파트너쉽이 어떻게 된거죠?"
reuslt_4 = qa_bot({"question": question_4, "chat_history":chat_history})
chat_history.extend([(question_4, reuslt_4["answer"])])

Llama.generate: prefix-match hit


 What is the nature of Samsung's partnerships mentioned in the article?

Llama.generate: prefix-match hit


 The article mentions Samsung partnering with various companies such as Microsoft, Google, and Qualcomm. These partnerships seem to be focused on developing and releasing co-branded or integrated products and services. For example, Samsung and Microsoft have partnered on Windows Phone devices and Samsung Knox security platform. Similarly, Samsung and Google have partnered on various Android devices and services such as Google Play Services and Samsung Health. Overall, these partnerships seem to be strategic alliances aimed at enhancing Samsung's product offerings and competitiveness in the marketplace.

In [25]:
reuslt_4["answer"]

" The article mentions Samsung partnering with various companies such as Microsoft, Google, and Qualcomm. These partnerships seem to be focused on developing and releasing co-branded or integrated products and services. For example, Samsung and Microsoft have partnered on Windows Phone devices and Samsung Knox security platform. Similarly, Samsung and Google have partnered on various Android devices and services such as Google Play Services and Samsung Health. Overall, these partnerships seem to be strategic alliances aimed at enhancing Samsung's product offerings and competitiveness in the marketplace."

In [27]:
answer_4_ko = ko_trans(reuslt_4['answer'])
answer_4_ko

'이 기사에는 Samsung이 Microsoft, Google 및 Qualcomm과 같은 다양한 회사와 파트너 관계를 맺고 있습니다.이러한 파트너십은 공동 브랜드 또는 통합 제품 및 서비스 개발 및 공개에 중점을 둔 것 같습니다.예를 들어, Samsung과 Microsoft는 Windows Phone 장치 및 Samsung Knox Security 플랫폼에서 파트너 관계를 맺었습니다.마찬가지로 Samsung과 Google은 Google Play Services 및 Samsung Health와 같은 다양한 Android 장치 및 서비스에서 파트너 관계를 맺었습니다.전반적으로, 이러한 파트너십은 시장에서 삼성의 제품 제공과 경쟁력을 향상시키기위한 전략적 제휴 인 것 같습니다.'

In [28]:
question_5 = "이러한 파트너쉽에 따라 예측 되는 반도체 산업의 추세는?"
reuslt_5 = qa_bot({"question": question_5, "chat_history":chat_history})
chat_history.extend([(question_5, reuslt_5["answer"])])

Llama.generate: prefix-match hit


 What is the predicted trend for the semiconductor industry based on these partnerships?

Llama.generate: prefix-match hit


 Based on these partnerships, it appears that there is a growing trend towards collaboration and consolidation in the semiconductor industry. This trend is likely driven by several factors, including increasing competition and pressure to innovate and reduce costs. As a result, it is likely that we will continue to see more partnerships and consolidations in the semiconductor industry in the coming years.

In [30]:
reuslt_5['answer']

' Based on these partnerships, it appears that there is a growing trend towards collaboration and consolidation in the semiconductor industry. This trend is likely driven by several factors, including increasing competition and pressure to innovate and reduce costs. As a result, it is likely that we will continue to see more partnerships and consolidations in the semiconductor industry in the coming years.'

In [31]:
answer_5_ko = ko_trans(reuslt_5['answer'])
answer_5_ko

'이러한 파트너십을 바탕으로 반도체 산업에서 협업 및 통합에 대한 추세가 증가하고있는 것으로 보입니다.이러한 추세는 경쟁 증가 및 비용을 혁신하고 절감 해야하는 압력을 포함하여 몇 가지 요인에 의해 주도 될 것입니다.결과적으로 앞으로 몇 년 동안 반도체 산업에서 더 많은 파트너십과 통합을 계속 볼 수있을 것입니다.'

In [32]:
question_6 = "만약 주식을 한다면 반도체 산업에 투자하는게 좋을까?"
reuslt_6 = qa_bot({"question": question_6, "chat_history":chat_history})
chat_history.extend([(question_6, reuslt_6["answer"])])

Llama.generate: prefix-match hit


 Should I invest in the semiconductor industry if I am interested in stocks?

Llama.generate: prefix-match hit


 The semiconductor industry has been a significant contributor to technological advancements and economic growth in recent decades. The industry is expected to continue its growth trajectory due to increasing demand for semiconductors in various industries such as automotive, consumer electronics, industrial automation, and telecommunications. Furthermore, advancements in semiconductor technology such as the development of smaller and more powerful semiconductors using advanced manufacturing processes such as FinFET and EUV lithography are expected to drive further growth and innovation in the semiconductor industry. Therefore, if you are interested in investing in stocks and are considering investing in the semiconductor industry, it may be a good investment opportunity due to the industry's strong growth trajectory and its significant contribution to technological advancements and economic growth in various industries. However, as with any investment opportunity, it is important to c

In [33]:
reuslt_6['answer']

" The semiconductor industry has been a significant contributor to technological advancements and economic growth in recent decades. The industry is expected to continue its growth trajectory due to increasing demand for semiconductors in various industries such as automotive, consumer electronics, industrial automation, and telecommunications. Furthermore, advancements in semiconductor technology such as the development of smaller and more powerful semiconductors using advanced manufacturing processes such as FinFET and EUV lithography are expected to drive further growth and innovation in the semiconductor industry. Therefore, if you are interested in investing in stocks and are considering investing in the semiconductor industry, it may be a good investment opportunity due to the industry's strong growth trajectory and its significant contribution to technological advancements and economic growth in various industries. However, as with any investment opportunity, it is important to 

In [34]:
answer_6_ko = ko_trans(reuslt_6['answer'])
answer_6_ko

'반도체 산업은 최근 수십 년 동안 기술 발전과 경제 성장에 크게 기여해 왔습니다.이 산업은 자동차, 소비자 전자 장치, 산업 자동화 및 통신과 같은 다양한 산업에서 반도체에 대한 수요가 증가함에 따라 성장 궤적을 계속할 것으로 예상됩니다.또한, Finfet 및 EUV 리소그래피와 같은 고급 제조 공정을 사용하는 더 작고 강력한 반도체의 개발과 같은 반도체 기술의 발전은 반도체 산업에서 더 많은 성장과 혁신을 주도 할 것으로 예상됩니다.따라서 주식 투자에 관심이 있고 반도체 산업에 대한 투자를 고려하고 있다면 산업의 강력한 성장 궤적과 다양한 산업의 기술 발전 및 경제 성장에 대한 상당한 기여로 인해 좋은 투자 기회가 될 수 있습니다.그러나 모든 투자 기회와 마찬가지로 투자 결정을 내리기 전에 업계 내 업계 및 특정 회사에 대한 철저한 연구 및 분석을 수행하는 것이 중요합니다.또한 투자 결정을 내리기 전에 투자 목표, 위험 내성 및 투자 지평을 고려하는 것이 중요합니다.'

In [35]:
chat_history

[('무슨 내용에 관한 뉴스입니까?',
  ' Yes, it appears to be a news article based on its format and content.'),
 ('Summarize the content of the news',
  ' This news article discusses a company called "Samsung Electronics Co., Ltd." and its plans for a new type of memory called "High Bandwidth Memory (HBM)" for use in graphics processing units (GPUs) and other high-performance computing applications. The article also mentions other types of memory such as "DRAM (Dynamic Random Access Memory)" and "HBM2 (High Bandwidth Memory version 2)" and their respective uses and advantages in various computing applications. The article also provides some background information on Samsung Electronics and its role in the semiconductor industry. Overall, the article provides valuable information on Samsung Electronics and its plans for High Bandwidth Memory (HBM) for use in high-performance computing applications such as graphics processing units (GPUs). The article also provides some background information on Sams

In [38]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [60]:
def get_web_list(web_path):
  web_lsit = []
  web_list.append(web_path)
  return web_list

In [ ]:
web_list =

In [ ]:
def get_text(web):

    doc_list = []

    for doc in docs:
        file_name = doc.name  # doc 객체의 이름을 파일 이름으로 사용
        with open(file_name, "wb") as file:  # 파일을 doc.name으로 저장
            file.write(doc.getvalue())
            logger.info(f"Uploaded {file_name}")
        if '.pdf' in doc.name:
            loader = PyPDFLoader(file_name)
            documents = loader.load_and_split()
        elif '.docx' in doc.name:
            loader = Docx2txtLoader(file_name)
            documents = loader.load_and_split()
        elif '.pptx' in doc.name:
            loader = UnstructuredPowerPointLoader(file_name)
            documents = loader.load_and_split()
        if 'news.naver' in doc.name:
              loader = WebBaseLoader(docs,
                                     bs_kwargs=dict(
                                         parse_only=bs4.SoupStrainer(
                                             class_=("media_end_head_title", "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME", "newsct_article _article_body"))),)
              documents = loader.load()


        doc_list.extend(documents)
    return doc_list

In [70]:
def get_text(web_path):
  doc_list = []
  if 'news.naver' in web_path:
    loader = WebBaseLoader(web_path,
                           bs_kwargs=dict(
                               parse_only=bs4.SoupStrainer(
                                   class_=("media_end_head_title", "media_end_head_info_datestamp_time _ARTICLE_DATE_TIME", "newsct_article _article_body"))))
    documents = loader.load()
    doc_list.extend(documents)
  return doc_list

In [71]:
file_path = "https://n.news.naver.com/mnews/article/003/0012345220?sid=101"

In [72]:
get_text(file_path)

[Document(page_content='\n"전반전 끝난 실적 시즌"…2차전지·조선·화학 부진\n2024.01.29. 오후 3:28\n\n"1분기 컨센서스는 상향 조정 중""올해 이익 증감은 반도체가 결정"\n\n\n\n[서울=뉴시스] 박은비 기자 = 국내 상장회사들의 지난해 4분기 실적 발표가 후반부로 접어든 가운데 지금까지 공개된 기업 중에는 2차전지, 조선, 화학 업종 실적이 부진한 것으로 나타났다.29일 유안타증권에 따르면 이 증권사가 집계하는 200개 종목의 지난해 4분기 실적 전망치 달성률은 72.4%, 어닝서프라이즈 비율은 27.3% 수준이다.소재(33.3%), 필수소비재(71.6%), 정보기술(IT)(72.2%) 섹터 부진이 증시 전체 달성률을 밑돌았다. 업종 기준으로는 2차전지(적자 전환), 조선(24.8%), 화학(35.1%) 부진이 두드러졌다.이로 인해 지난해 4분기 이익전망치는 실적을 반영하며 지속적으로 하향 조정 중이다. 조창민 유안타증권 연구원은 "지난해 4분기 이익전망치는 현재 33조5000억원으로 4주 연속 -5% 이상 조정이 진행 중"이라며 "특징적인 부분은 이와는 다르게 올해 1분기 컨센서스는 상향 조정되고 있다는 점"이라고 말했다.그러면서 "증시 전망치 달성률에 크게 영향을 주는 대형주들의 실적 발표가 대부분 완료됐다는 점도 변동성을 낮추는 요인"이라며 "실적 발표를 하지 않은 156개 종목들 중 증시 대비 이익 비중이 1%를 초과하는 종목은 18개, 2% 초과 종목은 7개에 불과하다"고 언급했다.이경민 대신증권 FICC부장은 "코스피의 차별적인 급락을 야기했던 코스피 실적 불안 심리는 SK하이닉스 실적 서프라이즈, 반도체 실적 전망 상향 조정을 계기로 진정됐다"며 "추세 반전을 논하기는 어려운 시점이지만 당분간 급락세 진정에 이은 코스피 반등 시도는 충분히 가능하다고 본다"고 설명했다.올해 국내 기업 이익 증감은 반도체가 결정할 것이라는 게 증권가 시각이다. 이재만 하나증권 연구원은 "삼성전자를 포함한 반도체 영업이익은 

In [55]:
def load_db(file):
    # split documents
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(doc_list)

    # define embedding
    embeddings =  HuggingFaceEmbeddings(
                                        model_name="snunlp/KR-SBERT-V40K-klueNLI-augSTS",
                                        model_kwargs={'device': 'cpu'},
                                        encode_kwargs={'normalize_embeddings': True}
    )

    # create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)

    # define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

    return retriever

In [56]:
def get_conversation_chain(retriever):
    qa = ConversationalRetrievalChain.from_llm(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )

    return qa

In [ ]:
def get_answer(qa, query, chat_history):
  result =

In [46]:
file_path = "https://n.news.naver.com/mnews/article/003/0012345220?sid=101"

qa_bot = load_db_2(file_path)

In [48]:
question_1 = "뉴스 내용을 요약해줘"
reuslt_1 = qa_bot({"question": question_1, "history": history})
#chat_history.extend([(question_6, reuslt_6["answer"])])

 The news article

Llama.generate: prefix-match hit


 discusses a decline in profits for certain industries such as semiconductors, chemicals, and shipping. The article also mentions that some companies in these industries have experienced large losses and have had to convert from a loss position to a profit position. The article also mentions that some analysts believe that the profits for these industries will increase significantly in the future due to factors such as increased demand for semiconductors and other high-tech products, as well as increased demand for chemicals and other raw materials due to economic growth and population growth in developing countries such as China and India. The article also mentions that some analysts believe that the profits for these industries will be negatively impacted in the future due to factors such as increased competition from other companies in these industries and from companies in other industries that are able to produce similar products more cheaply due to economies of scale or other cos

ValueError: One output key expected, got dict_keys(['answer', 'source_documents', 'generated_question'])

In [49]:
reuslt_1['answer']

' Yes, it appears to be a news article based on its format and content.'

In [ ]:
question_2 = "내용을 조금 더 짧게 해줘"
reuslt_2 = qa_bot({"question": question_2})
#chat_history.extend([(question_6, reuslt_6["answer"])])

In [ ]:
import panel as pn
import param

class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.loaded_file = "docs/cs229_lectures/MachineLearning-Lecture01.pdf"
        self.qa = load_db(self.loaded_file,"stuff", 4)

    def call_load_db(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        self.chat_history.extend([(query, result["answer"])])
        self.db_query = result["generated_question"]
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, style={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    @param.depends('db_query ', )
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    def get_sources(self):
        if not self.db_response:
            return
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history')
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    def clr_history(self,count=0):
        self.chat_history = []
        return


In [ ]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
dashboard

In [42]:
from langchain.vectorstores import Chroma

--------------------------------------

In [54]:
# cleanup
db.delete_collection()

NameError: name 'db' is not defined